In [1]:


from tqdm import tqdm
import numpy as np
import pandas as pd
import datatable as dt
import dask.dataframe as dd

from copy import copy

from pathlib import Path

from time import time, sleep
from contextlib import contextmanager

import matplotlib.pyplot as plt
import plotly.express as plx
import seaborn as sns

import os, sys

import gc

# print(plt.style.available)
plt.style.use("dark_background")
# plt.style.use("seaborn-dark")

# cluster = LocalCluster()
# client = Client(cluster)


print(f"curdir {Path.cwd()}")

curdir /home/sergey/mnt/st1500/Usr/Sergey/TheJob/Challenges/2020Riiid


In [2]:
%%time

dtypes = {
    "row_id": "int64",
    # "timestamp": "int64",
    "user_id": "int32",
    "content_id": "int16",
    # "content_type_id": "boolean",
    # "task_container_id": "int16",
    # "user_answer": "int8",
    "answered_correctly": "int8",
    "prior_question_elapsed_time": "float32",
    "prior_question_had_explanation": "boolean"
}


#     data_pd = pd.read_csv("./input/train.csv", dtype=dtypes, nrows=12.5e6, low_memory=False)
# data_pd = pd.read_csv("./input/train.csv", usecols=dtypes.keys(),  dtype=dtypes, low_memory=False)
# data_dd = dd.from_pandas(data_pd, chunksize=int(1e4))
# del data_pd
# _ = gc.collect()
data_pd = pd.read_feather("/mnt/data30G/2020riid/riiid_train.feather")
# train_dd = dd.from_pandas(train, chunksize=int(1e4))
# train_dd = dd.read_csv("/mnt/data30G/2020riid/train.csv",
#                                dtype=dask_dtypes,
#                                assume_missing=True,
#                                low_memory=False,
#                       )


# train = train.loc[:1e5]


CPU times: user 3.07 s, sys: 3.74 s, total: 6.81 s
Wall time: 23.3 s


In [3]:
data_pd.memory_usage()

Index                                   128
row_id                            809842656
timestamp                         809842656
user_id                           404921328
content_id                        202460664
content_type_id                   101230332
task_container_id                 202460664
user_answer                       101230332
answered_correctly                101230332
prior_question_elapsed_time       404921328
prior_question_had_explanation    809842656
dtype: int64

In [4]:
data_pd.info

<bound method DataFrame.info of               row_id  timestamp     user_id  content_id  content_type_id  \
0                  0          0         115        5692            False   
1                  1      56943         115        5716            False   
2                  2     118363         115         128            False   
3                  3     131167         115        7860            False   
4                  4     137965         115        7922            False   
...              ...        ...         ...         ...              ...   
101230327  101230327  428564420  2147482888        3586            False   
101230328  101230328  428585000  2147482888        6341            False   
101230329  101230329  428613475  2147482888        4212            False   
101230330  101230330  428649406  2147482888        6343            False   
101230331  101230331  428692118  2147482888        7995            False   

           task_container_id  user_answer  answered_cor

In [5]:
%%time
print("groupby started")
users = data_pd.groupby("user_id")

groupby started
CPU times: user 616 µs, sys: 0 ns, total: 616 µs
Wall time: 452 µs


In [6]:
users_len = len(users)
nvalid = int(users_len/5)
print("len(users)", users_len, nvalid)

len(users) 393656 78731


In [ ]:
%%time

print("users.tail(nvalid) started")
valid_pd = users.tail(nvalid)
valid_pd.reset_index().to_feather("/mnt/data30G/2020riid/valid_small.feather")
valid_mean = valid_pd.answered_correctly.dropna().values.mean()

del valid_pd
_ = gc.collect()

users.tail(nvalid) started


In [ ]:
%%time

print("users.head(users_len-nvalid) started")
train_pd = users.head(users_len-nvalid)
train_pd.reset_index().to_feather("/mnt/data30G/2020riid/train_small.feather")
train_mean = train_pd.answered_correctly.dropna().values.mean()
print("train_pd", train_pd.shape)

del train_pd
_ = gc.collect()

# print("valid_pd", valid_pd.shape)

In [ ]:
print(" trainmean validmean")
print(train_mean, valid_mean)



In [ ]:
# with timer("types SSD"):
#     for col, type_ in dask_dtypes.items():
#         print(col, train_dd[col].dtype())
#         # train[col] = train[col].astype(type_)
#         train_dd[col] = delayed(train_dd[col].astype(type_))

In [ ]:
%%time
# train_dd = train_dd.loc[:10000]
print("1 train_dd.head()")

# train_dd.head()
print(type(train_dd))
train_dd.shape[0].compute()

In [ ]:
%%time

if "row_id" in train_dd.columns:
    # train.set_index("row_id", inplace=True)
    train_dd = train_dd.set_index("row_id")
    # res = train_dd.mean()
    # res = delayed(train_dd.mean)()
# res_delayed = res.compute()
print("res = train_dd.mean()")
# print(res_delayed)

In [ ]:
%%time

res_delayed = delayed(train_dd.mean)()


In [ ]:
res_delayed.visualize()

In [ ]:
%%time

with ProgressBar():
    out = res_delayed.compute()

    print(f" out res_delayed = \n {out}")

In [ ]:
res_delayed.visualize()

In [ ]:
# train_dd["attempt"] = -1

print(f" train_dd.head() 2 = ")
train_dd.head()


In [ ]:
train_dd.count().compute()

In [ ]:
len(train_dd) - train_dd.count().compute()

In [ ]:
print(train_dd.shape[0].compute(), train_dd.shape[1])

In [ ]:
pd.Series(train_dd.user_id.compute()).nunique()


In [ ]:
%%time
print("max", train_dd.user_id.compute().value_counts().max())
print("mean", train_dd.user_id.compute().value_counts().mean())
print("median", train_dd.user_id.compute().value_counts().quantile(.5))

In [ ]:
%%time
max, mean, median = compute(
    train_dd.user_id.value_counts().max(),
    train_dd.user_id.value_counts().mean(),
    train_dd.user_id.value_counts().quantile(.5)
)
print(max, mean, median)

In [ ]:
def get_user_quantiles(df, plot=False):

    quantiles = []
    for q in np.arange(start=.4, stop=.8, step=.1):
        quantiles.append((q, df.user_id.value_counts().quantile(q=q).compute()))
        if plot:
            print(f"quantile {quantiles[-1]}")
            plt.plot(*quantiles[-1], marker="o",  c="w")
    return quantiles

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
train_dd.columns

In [ ]:
# total = train_dd.user_id.nunique()
groups = train_dd.groupby(by="user_id")
# total = len(groups.size())
# print("total", total)

In [ ]:
%%time
def print_count(group):
    print("print 1 from print_count")

groups_apply = groups.sum()

In [ ]:
%%time
groups_apply.compute()

In [ ]:
iter = groups


In [ ]:
%%time

content_id_sum = groups.content_id.sum()

print("content_id_sum.compute()")
content_id_sum.compute()

In [ ]:
%%time
agg = groups.aggregate(['sum', 'mean', 'max', 'min', list]).compute()

In [ ]:
agg

In [ ]:
agg.loc[115, "timestamp"]["sum"]

In [ ]:
with timer("rolling values"):
    for id, df in tqdm(groups, total=total):
        print("ok")
        # print(f"  user_id = {id}  shape = {df.shape[0]} is_monotonic {df.timestamp.is_monotonic}")
        pass
        # train_dd.loc[df.index, "attempt"] = range(1, df.shape[0]+1)

        # quantiles = get_user_quantiles(df=df)
        #
        # for quantile, val in quantiles:
        #     train_dd.loc[df.index, f"q{quantile}_None"] = df.timestamp.rolling(
        #         int(val),
        #         min_periods=1,
        #         win_type=None,
        #     ).quantile(quantile)
        #
        #     train_dd.loc[df.index, f"q{quantile}_gauss"] = df.timestamp.rolling(
        #         int(val),
        #         min_periods=1,
        #         win_type="gaussian",
        #     ).mean(std=3)
        #
        #     train_dd.loc[df.index, f"q{quantile}_exp"] = df.timestamp.rolling(
        #         int(val),
        #         min_periods=1,
        #         win_type="exponential",
        #     ).sum(tau=3)

# train_dd = train_dd.compute()



In [ ]:
train_shuffled = train.sample()
print("train.shape", train.shape)
train.head(20)

In [ ]:
submit_example = pd.read_csv("./input/example_sample_submission.csv", )
test = pd.read_csv("./input/example_test.csv")
lectures = pd.read_csv("./input/lectures.csv")
questions = pd.read_csv("./input/questions.csv")
print("ok")

In [ ]:
print("submit_example.shape", submit_example.shape[0])
submit_example.head()

In [ ]:
print("test.shape", test.shape)
test.head()

In [ ]:
print("lectures.shape", lectures.shape)
lectures.head()



In [ ]:
set(train.columns).symmetric_difference(set(test.columns))

In [ ]:
set(test.columns) - set(train.columns)

In [ ]:
set(train.columns) - set(test.columns)

In [ ]:
train.columns

In [ ]:
test.columns


In [ ]:
%time

# train_dt = dt.fread("./input/train.csv")
# print("train_dt.head()")
# train_dt.head(5)

##Questions

In [ ]:
print("questions.shape", questions.shape)
questions.head()

In [ ]:
bundle_ids = questions.value_counts(subset=["bundle_id"])
print(bundle_ids)
plt.hist(x=bundle_ids, bins=10);


In [ ]:
questions.value_counts(subset=["correct_answer"])


In [ ]:
questions.value_counts(subset=["part"])



In [ ]:
questions.tags = questions.tags.astype(str).apply([str.strip, str.split])
questions.tags[0][0]

In [ ]:
# ll = train_dt.shape[0]
#
# uniques = dt.unique(train_dt["user_id"]).shape[0]
# print(ll)
# print(uniques)
# print(ll/uniques)

In [ ]:
value_counts = train.user_id.value_counts()

In [ ]:
value_counts

In [ ]:
plt.hist(value_counts);